In [1]:
import glob
import SimpleITK as sitk
import numpy as np
import collections
from PIL import Image, ImageDraw
from segmentation import segment, init_segment_model
import os
import csv
import collections
from collections import namedtuple 
import random

In [2]:
# mhd_list = glob.glob('D:/LUNA/Luna16_SegData/subset*/*.mhd')
mhd_list = glob.glob('D:/LUNA/Luna16_AugData/subset*/*.mhd')
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, diameter_mm, series_uid, center_xyz, classes')
requireOnDisk_bool=True
presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

candidateInfo_list = []
with open('C:/Users/oplab/Desktop/Luna16_data/annotations_with_malignancy.csv', "r") as f:
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]
        annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
        annotationDiameter_mm = float(row[4])
        isMal_bool = {'False': False, 'True': True}[row[5]] #it record the malignancy or not
        classes = 1 if isMal_bool else 2

        if series_uid not in presentOnDisk_set and requireOnDisk_bool:
            continue

        candidateInfo_list.append(
            CandidateInfoTuple(
                True,
                True,
                isMal_bool,
                annotationDiameter_mm,
                series_uid,
                annotationCenter_xyz,
                classes
            )
        )

In [3]:
random.seed(42)
random.shuffle(candidateInfo_list)

In [4]:
print(len(candidateInfo_list))

1182


In [5]:
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

In [6]:
def np2MarkPng(np_arr, target_path, row, col):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    draw = ImageDraw.Draw(slices)
    x1, y1, x2, y2 = col-10, row-10, col+10, row+10
    draw.rectangle([x1, y1, x2, y2], outline="white", width=1)
    slices.save(target_path)

In [7]:
def np2Png(np_arr, target_path):
    min_value = np.min(np_arr)
    max_value = np.max(np_arr)
    scaled_np_arr = (np_arr - min_value) / (max_value - min_value) * 255
    scaled_np_arr = scaled_np_arr.astype(np.uint8)
    slices = Image.fromarray(scaled_np_arr, mode='L')
    slices.save(target_path)

In [25]:
# subdir_ori = ["segment", "segment_aug1", "segment_aug2"]
# subdir_mark = ["segment_mark", "segment_mark_aug1", "segment_mark_aug2"]
subdir_ori = ["origin", "origin_aug1", "origin_aug2"]
subdir_mark = ["segment_mark", "origin_mark_aug1", "origin_mark_aug2"]
# temp_a = np.zeros((2, 512, 512))
# temp_irc = np.zeros((2, 3))
for i, candidateInfo_tuple in enumerate(candidateInfo_list):
    # mhd_path = glob.glob('D:/LUNA/Luna16_SegData/subset*/{}.mhd'.format(candidateInfo_tuple.series_uid))
    mhd_path = glob.glob('D:/LUNA/Luna16_AugData/subset*/{}.mhd'.format(candidateInfo_tuple.series_uid))

    ct_mhd = sitk.ReadImage(mhd_path)

    if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
        ct_mhd = ct_mhd[...,0]
        
    hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)
    
    center_irc = xyz2irc(candidateInfo_tuple.center_xyz, origin_xyz, vxSize_xyz, direction_a)
    
    index, row, column = center_irc
    
    offset = 0
    
    j = 0
    
    model, device = init_segment_model()
    
#     temp_a[i%2, :, :] = hu_a[index+j]
    
#     temp_irc[i%2, :] = np.array([index, row, column])
    
#     if i % 2 == 1:
#         target = segment(temp_a, model, device)
        # np2Png(target[0], "D:/LUNA/yolo/{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_ori[offset], i - 1, j, candidateInfo_tuple.classes, subdir_ori[offset], temp_irc[0, 0], candidateInfo_tuple.series_uid))
        # np2MarkPng(target[0], "D:/LUNA/yolo/{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_mark[offset], i - 1, j, candidateInfo_tuple.classes, subdir_ori[offset], temp_irc[0, 0], candidateInfo_tuple.series_uid), temp_irc[0, 1], temp_irc[0, 2])
#         np2Png(target[1], "D:/LUNA/yolo/{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_ori[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], temp_irc[1, 0], candidateInfo_tuple.series_uid))
#         np2MarkPng(target[1], "D:/LUNA/yolo/{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_mark[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], temp_irc[1, 0], candidateInfo_tuple.series_uid), temp_irc[1, 1], temp_irc[1, 2])

    hu_a_expanded = np.expand_dims(hu_a[index+j], axis=0)
    target = segment(hu_a_expanded, model, device)
    target = np.squeeze(target)
    np2Png(hu_a[index+j], "D:/LUNA/yolo/{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_ori[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid))
    np2MarkPng(target, "D:/LUNA/yolo/{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_mark[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid), row, column)
    
    # for j in range(-2, 3):
    #     offset = (abs(j)) % 3
    #     # if candidateInfo_tuple.classes == 0:
    #         # np2Png(hu_a[index+j], "D:/LUNA/yolo/{}/non-nodule/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_ori[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid))
    #         # np2MarkPng(hu_a[index+j], "D:/LUNA/yolo/{}/non-nodule/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_mark[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid), row, column)
    #     # else:
    #     if offset == 1:
    #         np2Png(hu_a[index+j], "D:/LUNA/yolo/{}/nodule/offset_{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_ori[offset], j, i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid))
    #         np2MarkPng(hu_a[index+j], "D:/LUNA/yolo/{}/nodule/offset_{}/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_mark[offset], j, i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid), row, column)
    #     else:
    #         np2Png(hu_a[index+j], "D:/LUNA/yolo/{}/nodule/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_ori[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid))
    #         np2MarkPng(hu_a[index+j], "D:/LUNA/yolo/{}/nodule/{}-({})_class-{}_{}_idx-{}_{}.png".format(subdir_mark[offset], i, j, candidateInfo_tuple.classes, subdir_ori[offset], index, candidateInfo_tuple.series_uid), row, column)

KeyboardInterrupt: 

In [ ]:
# for i, candidateInfo_tuple in enumerate(candidateInfo_list):
#     # mhd_path = glob.glob('D:/LUNA/Luna16_SegData/subset*/{}.mhd'.format(candidateInfo_tuple.series_uid))
#     mhd_path = glob.glob('D:/LUNA/Luna16_AugData/subset*/{}.mhd'.format(candidateInfo_tuple.series_uid))

#     ct_mhd = sitk.ReadImage(mhd_path)

#     if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
#         ct_mhd = ct_mhd[...,0]
        
#     hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

#     origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

#     vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

#     direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)
    
#     center_irc = xyz2irc(candidateInfo_tuple.center_xyz, origin_xyz, vxSize_xyz, direction_a)
    
#     index, row, column = center_irc


# total_rate = 0
# total = (512 * 512)
# i = 0
# average_rate = 0
# for i in range(8001):
#     # if (i % 10 == 0):
#     #     # print("i = ", i)
#     #     average_rate += (total_rate) / 10
#     #     # print("avg = ", average_rate)
#     #     total_rate = 0
#     if (i % 1000 == 0):
#         print("i = ", i)
#         print("avg = ", average_rate)
#     i += 1
#     csum = (train_ds[i][1].sum())
#     # print("sum = ", csum)
#     total_rate += csum / total
# average_rate = (total_rate) / 8000
# print(average_rate)

In [ ]:
# print(center_irc)